In [1]:
import pandas as pd
task_df =  pd.read_csv('C:/Users/Arwa/Desktop/datasets/Ex2/Jobs_dataset_tokenized_Ex2.csv')

In [2]:
print(task_df.loc[52,'Job_Title'])
print(task_df.loc[53,'Job_Title'])

Wordpress Expert install theme
install wordpress theme plugins two


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

if task_df.loc[52,'Job_Title']!= '' and task_df.loc[53,'Job_Title']!= '':
    data = [
        task_df.loc[52,'Job_Title'],
        task_df.loc[53,'Job_Title']
            ]

    # Vectorise the data
    vec = TfidfVectorizer()

    X = vec.fit_transform(data)  # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

    # Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
    S = cosine_similarity(X)
    Title_Sim = S[1,0]
else:
    Title_Sim = 0
print('similarity of titles:', Title_Sim)

similarity of titles: 0.5101490193104813


In [4]:
task_df.loc[53,'Required_Skills 5'] = 'Plgin'
skills_t1 = {'CSS', 'HTML', 'PHP', 'WordPress', 'Plgin'}
skills_t2 = {'CSS', 'HTML', 'PHP', 'WordPress', 'Plgin'}
skills_t1

{'CSS', 'HTML', 'PHP', 'Plgin', 'WordPress'}

In [5]:
skills_t2

{'CSS', 'HTML', 'PHP', 'Plgin', 'WordPress'}

In [6]:
for s2 in skills_t2:
     print("s2",s2)

s2 Plgin
s2 PHP
s2 WordPress
s2 HTML
s2 CSS


In [7]:

from ete3 import TreeNode,Tree
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def concept_similarity_measure_ex2(C1, C2):
    
    taxonomy = Tree("skills_small_taxonomy_tree_level_score.nw")
    # print("C1",C1,"\n","C2",C2)
    

    N1 = 0 # the distance from Concept 1 to the least common subsumer
    N2 = 0 # the distance from Concept 2 to the least common subsumer
    N = 0  # the distance from  the least common subsumer to the root
    """ -----------------L   he shortest path between the tow concepts------------------------------"""


    node1 = taxonomy.search_nodes(name=C1)
    node2 = taxonomy.search_nodes(name=C2)
    # print('node1', node1)
    # print('node2', node2)
    # if the skill is not found in the taxonomy
    if node1 == [] or node2 == []:
        # print('skills not in taxonomy')
        # print(C1, C2)
        data = [C1, C2]

        # Vectorise the data
        vec = TfidfVectorizer()

        X = vec.fit_transform(
            data)  # `X` will now be a TF-IDF representation of the data, the first row of `X` corresponds to the first sentence in `data`

        # Calculate the pairwise cosine similarities (depending on the amount of data that you are going to have this could take a while)
        S = cosine_similarity(X)
        similarity = S[0, 1]


        # print('simmmms',similarity)
        l1 = l2 = 1.0  # how much should it be

    else:
        node1 = node1[0]
        node2 = node2[0]
        common = node1.get_common_ancestor(node2)
        # print(common.is_root())
        # print("common is ",common.name)

        """ ------------------N the distance from root node to the least common subsumer-------------------"""
        root = taxonomy.get_tree_root()
        N = taxonomy.get_distance(common, root, topology_only=False)
        # print("N = the distance between the common ancestor", common.name, "AND  ROOT IS ", N)

        """ ----------------N1 the distance from Concept 1 to the least common subsumer--------------------"""
        N1 = taxonomy.get_distance(C1, common, topology_only=False)
        # print("N1 = the distance between",C1,"AND  ROOT IS ",N1)

        """ ----------------N1 the distance from Concept 2 to the least common subsumer--------------------"""
        N2 = taxonomy.get_distance(C2, common, topology_only=False)
        # print("N2 = the distance between",C2, "AND  ROOT IS ", N2)

        """ -------------------------------COMPUTE THE MEASURE FORMULA----------------------------------------"""

        similarity = (2*N)/(N1+N2+(2*N))

        
        l1 = node1.level_score
        l2 = node2.level_score
        similarity = round(similarity, 2)
#         print("similarity between ", C1,"and",C2, "is", similarity)
#         print("level score  ",C1,'  is', l1)
#         print("level score  ",C2,'  is', l2)
#         print("---------------------------------------------------")
    return similarity,l1,l2




In [8]:
import numpy as np
def is_nan(x):
    return (x is np.nan)

In [9]:
skills_t1 = ['CSS', 'HTML', 'PHP', 'WordPress', 'Plgin']
skills_t2 = ['Plgin', 'Website Design', 'WordPress','Java']
skills_t1.sort()
skills_t2.sort()
print(skills_t1)
print(skills_t2)

['CSS', 'HTML', 'PHP', 'Plgin', 'WordPress']
['Java', 'Plgin', 'Website Design', 'WordPress']


In [13]:
weight_list = []
sim_max = []
ave_list = [] 
i = 0
 
scaler = 0 
total_c = 0 
wve = 0
for s1 in skills_t1:
    c = 0 
    j = 0 
    i = i+1
    sim_list = []
    if not is_nan(s1):
        
        for s2 in skills_t2:
            
            if not is_nan(s2):
                print("s1", s1)
                print("s2",s2)
                sim ,l1, l2 = concept_similarity_measure_ex2(s1, s2) # this function returns a tuple (sim, l1,l2)
                sim = float(sim)  # convert to floats
                l1 = float(l2)
                l2 = float(l1)
                print(sim)
        
            
                sim_list.append(sim)
        print('sim_list of round #',i, "is  ", sim_list)
        sim_list = np.array(sim_list)
        
#         maxima = sim_list.argmax()
#         print("argmax = ", maxima)
                
        for k in sim_list:
            if k >= 0.8025:
                c= c + k 
                j = j +1
                total_c = total_c + 1 
        print("sum values > 0.8 = ", c)
        print("---------------------------------------------------")

        scaler = scaler + len(sim_list)  
        if j != 0:
            ave = round(c/j, 1)
        print("AVE of the lsit", ave)
        ave_list.append(ave)
print("total count of values > 0.8 = ", total_c)
skills_sim = round(total_c /scaler,1) 
print("length of all the lists ", scaler)
print("SKILLS SIM total count /length of list * number of rounds = ", skills_sim)
print("---------------------------------------------------")

print("average list", ave_list)
print("average of sim lsit",sum(ave_list)/len(sim_list))


s1 CSS
s2 Java
0.25
s1 CSS
s2 Plgin
0.55
s1 CSS
s2 Website Design
0.75
s1 CSS
s2 WordPress
0.6
sim_list of round # 1 is   [0.25, 0.55, 0.75, 0.6]
sum values > 0.8 =  0
---------------------------------------------------


NameError: name 'ave' is not defined

In [14]:
C1 = 0.3
C2 = 0.7
task_2_task_similarity_score = C1*Title_Sim + C2*skills_sim
task_2_task_similarity_score = round(task_2_task_similarity_score, 1)
print(task_2_task_similarity_score)

NameError: name 'skills_sim' is not defined

In [23]:
skills_t1 = ['CSS', 'HTML', 'PHP', 'WordPress', 'Plgin']
skills_t2 = ['Plgin', 'Website Design', 'WordPress','SQL']
# skills_t1.sort()
# skills_t2.sort()
print(skills_t1)
print(skills_t2)

['CSS', 'HTML', 'PHP', 'WordPress', 'Plgin']
['Plgin', 'Website Design', 'WordPress', 'SQL']


In [24]:
weight_list = []
sim_max = []
ave_list = [] 
i = 0
import pandas as pd  
scaler = 0 
total_c = 0 
sim_df = pd.DataFrame()
ave = 0 
for s1 in skills_t1:
    c = 0 
    j = 0 
    i = i+1
    sim_list = []
    if not is_nan(s1):
        
        for s2 in skills_t2:
            
            if not is_nan(s2):
                print("s1", s1)
                print("s2",s2)
                sim ,l1, l2 = concept_similarity_measure_ex2(s1, s2) # this function returns a tuple (sim, l1,l2)
                sim = float(sim)  # convert to floats
                l1 = float(l2)
                l2 = float(l1)
                print(sim)
                sim_df.loc[s1,s2] = sim
            
                sim_list.append(sim)
        print('sim_list of round #',i, "is  ", sim_list)
        sim_list = np.array(sim_list)
        
#         maxima = sim_list.argmax()
#         print("argmax = ", maxima)
                
        for k in sim_list:
            if k >= 0.8025:
                c= c + k 
                j = j +1
                total_c = total_c + 1 
        print("sum values > 0.8 = ", c)
        print("---------------------------------------------------")

        scaler = scaler + len(sim_list) 
        if j != 0  :
            ave = round(c/j, 1)
            print("AVE of the lsit", ave)
        ave_list.append(ave)
print("total count of values > 0.8 = ", total_c)
skills_sim = round(total_c /scaler,1) 
print("length of all the lists ", scaler)
print("SKILLS SIM total count /length of list * number of rounds = ", skills_sim)
print("---------------------------------------------------")

print("average list", ave_list)
print("average of sim lsit",sum(ave_list)/len(sim_list))



s1 CSS
s2 Plgin
0.55
s1 CSS
s2 Website Design
0.75
s1 CSS
s2 WordPress
0.6
s1 CSS
s2 SQL
0.25
sim_list of round # 1 is   [0.55, 0.75, 0.6, 0.25]
sum values > 0.8 =  0
---------------------------------------------------
s1 HTML
s2 Plgin
0.6
s1 HTML
s2 Website Design
0.86
s1 HTML
s2 WordPress
0.67
s1 HTML
s2 SQL
0.29
sim_list of round # 2 is   [0.6, 0.86, 0.67, 0.29]
sum values > 0.8 =  0.86
---------------------------------------------------
AVE of the lsit 0.9
s1 PHP
s2 Plgin
0.8
s1 PHP
s2 Website Design
0.86
s1 PHP
s2 WordPress
0.89
s1 PHP
s2 SQL
0.29
sim_list of round # 3 is   [0.8, 0.86, 0.89, 0.29]
sum values > 0.8 =  1.75
---------------------------------------------------
AVE of the lsit 0.9
s1 WordPress
s2 Plgin
0.91
s1 WordPress
s2 Website Design
0.75
s1 WordPress
s2 WordPress
1.0
s1 WordPress
s2 SQL
0.25
sim_list of round # 4 is   [0.91, 0.75, 1.0, 0.25]
sum values > 0.8 =  1.9100000000000001
---------------------------------------------------
AVE of the lsit 1.0
s1 Plgin
s2 P

In [25]:
sim_df

,Plgin,Website Design,WordPress,SQL
CSS,0.55,0.75,0.60,0.25
HTML,0.60,0.86,0.67,0.29
PHP,0.80,0.86,0.89,0.29
WordPress,0.91,0.75,1.00,0.25
Plgin,1.00,0.67,0.91,0.22


In [26]:
sim_df.shape

(5, 4)

In [31]:
from scipy.linalg import svd


U, S, V = svd(sim_df, full_matrices=False)
U.shape, S.shape, V.shape




((5, 4), (4,), (4, 4))

In [32]:
# reconstructing the original matrix 
np.dot(U.dot(np.diag(S)), V)

array([[0.55, 0.75, 0.6 , 0.25],
       [0.6 , 0.86, 0.67, 0.29],
       [0.8 , 0.86, 0.89, 0.29],
       [0.91, 0.75, 1.  , 0.25],
       [1.  , 0.67, 0.91, 0.22]])

In [42]:
print("U")
print(U)
print("\n\n")
print("S")
print(S[0])
print(S[1])
print(S[2])
print(S[3])
print(S[4])
print("\n\n")
print("V")
print(V)

U
[[-0.35826264 -0.44665345  0.2099729  -0.71208919]
 [-0.40211295 -0.57011472  0.17368125  0.30636737]
 [-0.4794394  -0.13167309 -0.28364323  0.54110332]
 [-0.49820434  0.34548079 -0.64831706 -0.31933139]
 [-0.48154268  0.58204393  0.65190315  0.06559481]]



S
3.1304979029220643
0.38334106800625367
0.08617648526031156
0.002473614472609641


IndexError: index 4 is out of bounds for axis 0 with size 4

In [33]:
new_S = S[0]
new_U = U[:, 0]
new_U.dot(new_S)



array([-1.12154045, -1.25881373, -1.50088405, -1.55962765, -1.50746834])

In [133]:
s[0] #highst singular value 

3.1304979029220643

In [160]:

import pandas as pd
import numpy as np 
from sklearn import linear_model
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import normalize

In [161]:
svd = TruncatedSVD(n_components = 3)
x = svd.fit(sim_df).transform(sim_df)

In [162]:
svd

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=5,
       random_state=None, tol=0.0)

In [163]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

svd = TruncatedSVD(n_components=5)
svd.fit(X)  


print(svd.explained_variance_ratio_)  

print(svd.explained_variance_ratio_.sum())  

print(svd.singular_values_) 

[3.27114526e-31 1.00000000e+00]
1.0000000000000007
[1.22888121 0.69989355]


In [159]:
print(svd.explained_variance_ratio_ ) 

[0. 1.]


In [125]:
def test_algorithms():
    svd_a = TruncatedSVD(30, algorithm="arpack")
    svd_r = TruncatedSVD(30, algorithm="randomized", random_state=42)
 
    Xa = svd_a.fit_transform(X)[:, :6]
    Xr = svd_r.fit_transform(X)[:, :6]
    assert_array_almost_equal(Xa, Xr, decimal=5)
 
    comp_a = np.abs(svd_a.components_)
    comp_r = np.abs(svd_r.components_)
    # All elements are equal, but some elements are more equal than others.
    assert_array_almost_equal(comp_a[:9], comp_r[:9])
    assert_array_almost_equal(comp_a[9:], comp_r[9:], decimal=2)

In [126]:
test_algorithms()

ValueError: k must be between 1 and min(A.shape), k=30